#### Imports

In [2]:
import os
import json
from tqdm import tqdm

from configs import ConfigPath
from data_preprocessing.text_splitter import TextSplitter
from utils.utils import read_json_file
from llms.embedding_model import EmbeddingModel
from knowledge_graph.loader import GraphLoader
from knowledge_graph.crud import GraphCrud
from configs.config import ConfigEnv
from knowledge_graph.connection import Neo4jConnection
from llms.llm import ChatModel
from retrieval.tools.vector_search_tool import VectorSearchTool


#### Initializations

In [3]:
# models
embedding_model = EmbeddingModel()
chat_model = ChatModel(provider="google", model_name="gemini-2.0-pro-exp-02-05")
llm = chat_model.llm

2025-03-07 00:41:29,584 [DEBUG] embedding_model - CUDA is available, using GPU
2025-03-07 00:41:51,511 [DEBUG] embedding_model - Embedding model initialized: neuml/pubmedbert-base-embeddings


In [3]:
# data
data = read_json_file(file_path=os.path.join(ConfigPath.RAW_DATA_DIR, "pqa_labeled.json"))  

# modules
# text_splitter = TextSplitter()
neo4j_connection = Neo4jConnection(uri=ConfigEnv.NEO4J_URI, 
                 user=ConfigEnv.NEO4J_USER,
                 password=ConfigEnv.NEO4J_PASSWORD,
                 database=ConfigEnv.NEO4J_DB)
# crud = GraphCrud(neo4j_connection=neo4j_connection)
# graph_loader = GraphLoader(text_splitter=text_splitter,
#                            embedding_model=embedding_model,
#                            crud=crud,
#                            data=data)

vector_search_tool = VectorSearchTool(
    llm=llm,
    embedding_model=embedding_model,
    neo4j_connection=neo4j_connection,
    return_direct=False
)


2025-03-06 22:55:37,668 [DEBUG] connection - Connection successful!


In [ ]:
# answer = vector_search_tool.invoke("Can tailored interventions increase mammography use among HMO women?")
# print(answer)

yes


In [4]:
from evalution.evaluation import Evaluator

evaluator = Evaluator(ground_truth_data=data, retriever=vector_search_tool)

2025-03-06 22:55:49,097 [INFO] evaluation - Generating results...
Generating results...: 100%|██████████| 1000/1000 [1:19:54<00:00,  4.79s/it]
2025-03-07 00:15:43,448 [INFO] evaluation - Generated 1000 results.
2025-03-07 00:15:43,493 [INFO] evaluation - Results saved to C:\Users\melis\Desktop\Projects\python_projects\thesis\advanced_graph_rag\data\results\evaluation_results_20250307_001543.json


In [5]:
evaluator.compute_metrics()

2025-03-07 00:15:50,392 [INFO] evaluation - Computing metrics...
2025-03-07 00:15:50,508 [INFO] evaluation - Accuracy: 0.6780
2025-03-07 00:15:50,511 [INFO] evaluation - Precision: 0.6423
2025-03-07 00:15:50,512 [INFO] evaluation - Recall: 0.9402
2025-03-07 00:15:50,514 [INFO] evaluation - F1: 0.7632
2025-03-07 00:15:50,517 [INFO] evaluation - Results saved to C:\Users\melis\Desktop\Projects\python_projects\thesis\advanced_graph_rag\data\metrics\answering_metrics_20250307_001550.json


{'accuracy': 0.678,
 'precision': 0.6423267326732673,
 'recall': 0.9402173913043478,
 'f1': 0.763235294117647}